## Dataset Preprocessing

In [ ]:
# Unified Dataset Preprocessor for Otsu and Hand labelled SEN1FLOODS11 dataset


!pip install rasterio

import os
import csv
import numpy as np
import rasterio
from PIL import Image
from tqdm import tqdm
import shutil
from skimage.transform import resize
import glob
from sklearn.model_selection import train_test_split
import random
import json

class UnifiedSARFloodPreprocessor:
    def __init__(self, otsu_path, sen1floods_path, output_path, img_size=256, 
                 test_size=0.2, val_size=0.2, random_seed=42):
        """
        Unified preprocessor for both S1F11-Otsu and Sen1Floods11 datasets
        
        Args:
            otsu_path: Path to S1F11-Otsu dataset
            sen1floods_path: Path to Sen1Floods11 dataset  
            output_path: Output directory for processed data
            img_size: Target image size (default 256)
            test_size: Test split ratio (default 0.2)
            val_size: Validation split ratio (default 0.2)
            random_seed: Random seed for reproducibility
        """
        self.otsu_path = otsu_path
        self.sen1floods_path = sen1floods_path
        self.output_path = output_path
        self.img_size = img_size
        self.test_size = test_size
        self.val_size = val_size
        self.random_seed = random_seed
        
        # Set random seed for reproducible splits
        random.seed(random_seed)
        np.random.seed(random_seed)
        
        # Initialize normalization parameters (calculated from combined training data)
        self.norm_means = None
        self.norm_stds = None
        
        # Dataset paths
        self.setup_dataset_paths()
        
        # Create output directories
        self.create_output_dirs()
        
        # Statistics for tracking combined dataset properties
        self.stats = {
            'train': {'count': 0, 'flood_pixels': 0, 'total_valid_pixels': 0, 'invalid_pixels': 0, 'otsu_count': 0, 'sen1floods_count': 0},
            'val': {'count': 0, 'flood_pixels': 0, 'total_valid_pixels': 0, 'invalid_pixels': 0, 'otsu_count': 0, 'sen1floods_count': 0},
            'test': {'count': 0, 'flood_pixels': 0, 'total_valid_pixels': 0, 'invalid_pixels': 0, 'otsu_count': 0, 'sen1floods_count': 0}
        }

    def setup_dataset_paths(self):
        """Setup paths for both datasets"""
        # Otsu dataset paths
        self.otsu_image_dir = os.path.join(self.otsu_path, 'S1Weak')
        self.otsu_label_dir = os.path.join(self.otsu_path, 'S1OtsuLabelWeak')
        
        # Sen1Floods11 dataset paths
        self.sen1floods_image_dir = os.path.join(self.sen1floods_path, 'data', 'flood_events', 'HandLabeled', 'S1Hand')
        self.sen1floods_mask_dir = os.path.join(self.sen1floods_path, 'data', 'flood_events', 'HandLabeled', 'LabelHand')
        self.sen1floods_splits_dir = os.path.join(self.sen1floods_path, 'splits', 'flood_handlabeled')

    def create_output_dirs(self):
        """Create output directory structure"""
        if os.path.exists(self.output_path):
            shutil.rmtree(self.output_path)
        
        for split in ['train', 'val', 'test']:
            os.makedirs(os.path.join(self.output_path, split, 'images'), exist_ok=True)
            os.makedirs(os.path.join(self.output_path, split, 'masks'), exist_ok=True)
            os.makedirs(os.path.join(self.output_path, split, 'validity_masks'), exist_ok=True)
            
        print(f"Created output directories at {self.output_path}")

    def find_otsu_pairs(self):
        """Find matching SAR images and labels for Otsu dataset"""
        print("Finding Otsu dataset pairs...")
        
        sar_files = glob.glob(os.path.join(self.otsu_image_dir, '*_S1Weak.tif'))
        label_files = glob.glob(os.path.join(self.otsu_label_dir, '*_S1OtsuLabelWeak.tif'))
        
        print(f"Found {len(sar_files)} Otsu SAR images")
        print(f"Found {len(label_files)} Otsu label files")
        
        # Create dictionaries for quick lookup
        sar_dict = {}
        label_dict = {}
        
        for sar_file in sar_files:
            basename = os.path.basename(sar_file)
            base_id = '_'.join(basename.split('_')[:-1])
            sar_dict[base_id] = sar_file
            
        for label_file in label_files:
            basename = os.path.basename(label_file)
            base_id = '_'.join(basename.split('_')[:-1])
            label_dict[base_id] = label_file
        
        # Find matching pairs
        matching_pairs = []
        for base_id in sar_dict:
            if base_id in label_dict:
                matching_pairs.append(('otsu', base_id, sar_dict[base_id], label_dict[base_id]))
            else:
                print(f"Warning: No Otsu label found for SAR image {base_id}")
        
        print(f"Found {len(matching_pairs)} Otsu matching pairs")
        return matching_pairs

    def read_sen1floods_csv(self, csv_path):
        """Read Sen1Floods11 CSV file"""
        if not os.path.exists(csv_path):
            print(f"Warning: CSV file not found: {csv_path}")
            return []
            
        with open(csv_path, 'r') as f:
            reader = csv.reader(f)
            next(reader, None)  # Skip header
            pairs = []
            for row in reader:
                if len(row) >= 2:
                    im_path = os.path.join(self.sen1floods_image_dir, row[0])
                    mask_path = os.path.join(self.sen1floods_mask_dir, row[1])
                    pairs.append(('sen1floods', f"sen1floods_{len(pairs)}", im_path, mask_path))
            return pairs

    def get_all_dataset_pairs(self):
        """Get all pairs from both datasets"""
        print("Collecting all dataset pairs...")
        
        # Get Otsu pairs
        otsu_pairs = self.find_otsu_pairs()
        
        # Get Sen1Floods11 pairs
        sen1floods_pairs = []
        for split_name in ['flood_train_data.csv', 'flood_val_data.csv', 'flood_test_data.csv']:
            csv_path = os.path.join(self.sen1floods_splits_dir, split_name)
            pairs = self.read_sen1floods_csv(csv_path)
            sen1floods_pairs.extend(pairs)
        
        all_pairs = otsu_pairs + sen1floods_pairs
        
        print(f"Total dataset pairs:")
        print(f"  Otsu: {len(otsu_pairs)} pairs")
        print(f"  Sen1Floods11: {len(sen1floods_pairs)} pairs")
        print(f"  Combined: {len(all_pairs)} pairs")
        
        return all_pairs

    def create_unified_splits(self, all_pairs):
        """Create unified train/val/test splits from combined datasets"""
        print(f"Creating unified splits with test_size={self.test_size}, val_size={self.val_size}")
        
        # Separate by dataset type for balanced splitting
        otsu_pairs = [p for p in all_pairs if p[0] == 'otsu']
        sen1floods_pairs = [p for p in all_pairs if p[0] == 'sen1floods']
        
        # Split each dataset separately to ensure representation
        if len(otsu_pairs) > 0:
            otsu_train_val, otsu_test = train_test_split(
                otsu_pairs, test_size=self.test_size, random_state=self.random_seed
            )
            adjusted_val_size = self.val_size / (1 - self.test_size)
            otsu_train, otsu_val = train_test_split(
                otsu_train_val, test_size=adjusted_val_size, random_state=self.random_seed
            )
        else:
            otsu_train = otsu_val = otsu_test = []
        
        if len(sen1floods_pairs) > 0:
            sen1floods_train_val, sen1floods_test = train_test_split(
                sen1floods_pairs, test_size=self.test_size, random_state=self.random_seed
            )
            adjusted_val_size = self.val_size / (1 - self.test_size)
            sen1floods_train, sen1floods_val = train_test_split(
                sen1floods_train_val, test_size=adjusted_val_size, random_state=self.random_seed
            )
        else:
            sen1floods_train = sen1floods_val = sen1floods_test = []
        
        # Combine splits
        splits = {
            'train': otsu_train + sen1floods_train,
            'val': otsu_val + sen1floods_val,
            'test': otsu_test + sen1floods_test
        }
        
        print(f"Unified split sizes:")
        for split_name, pairs in splits.items():
            otsu_count = len([p for p in pairs if p[0] == 'otsu'])
            sen1floods_count = len([p for p in pairs if p[0] == 'sen1floods'])
            total = len(pairs)
            percentage = 100 * total / len(all_pairs)
            print(f"  {split_name.upper()}: {total} samples ({percentage:.1f}%) - Otsu: {otsu_count}, Sen1Floods11: {sen1floods_count}")
        
        return splits

    def compute_bands(self, vh, vv):
        """
        Compute the three bands according to the specified formulas:
        Band 1: VV
        Band 2: NewBand1 = (VH - VV) / (VH + VV)
        Band 3: NewBand2 = sqrt((VH^2 + VV^2) / 2)
        """
        eps = 1e-8
        
        # Band 1: VV
        band1 = vv
        
        # Band 2: NewBand1 = (VH - VV) / (VH + VV)
        band2 = np.divide(vh - vv, vh + vv + eps)
        
        # Band 3: NewBand2 = sqrt((VH^2 + VV^2) / 2)
        band3 = np.sqrt((vh**2 + vv**2) / 2)
        
        return band1, band2, band3

    def process_image_for_stats(self, im_path):
        """Process image to collect statistics (first pass)"""
        try:
            with rasterio.open(im_path) as src:
                if src.count < 2:
                    print(f"Warning: Image {im_path} has only {src.count} band(s). Expected at least 2 (VH, VV)")
                    return None
                
                # Read VH and VV bands
                vh = src.read(1)
                vv = src.read(2)
                
                # Handle NaN and infinite values
                vh = np.nan_to_num(vh)
                vv = np.nan_to_num(vv)
                
                if vh.size == 0 or vv.size == 0:
                    print(f"Warning: Empty arrays in {im_path}")
                    return None
                
                # Compute the three bands
                band1, band2, band3 = self.compute_bands(vh, vv)
                
                # Create 3-channel image
                arr_x = np.stack([band1, band2, band3], axis=0)
                
                # Clip extreme values
                for i in range(3):
                    v_min, v_max = np.percentile(arr_x[i], [1, 99])
                    arr_x[i] = np.clip(arr_x[i], v_min, v_max)
                
                # Resize to target dimensions
                arr_x = np.stack([
                    resize(arr_x[0], (self.img_size, self.img_size), preserve_range=True),
                    resize(arr_x[1], (self.img_size, self.img_size), preserve_range=True),
                    resize(arr_x[2], (self.img_size, self.img_size), preserve_range=True)
                ], axis=0)
                
                return arr_x
                
        except Exception as e:
            print(f"Error processing image for stats {im_path}: {str(e)}")
            return None

    def calculate_normalization_params(self, train_pairs):
        """Calculate mean and std from combined training data"""
        print("Calculating normalization parameters from combined training data...")
        
        # Collect all pixel values for each band
        all_pixels = [[] for _ in range(3)]
        
        for dataset_type, base_id, im_path, _ in tqdm(train_pairs, desc="Collecting statistics"):
            if not os.path.exists(im_path):
                continue
                
            arr_x = self.process_image_for_stats(im_path)
            if arr_x is not None:
                for i in range(3):
                    all_pixels[i].append(arr_x[i].flatten())
        
        # Calculate means and stds
        means = []
        stds = []
        
        for i in range(3):
            if all_pixels[i]:
                combined_pixels = np.concatenate(all_pixels[i])
                means.append(np.mean(combined_pixels))
                stds.append(np.std(combined_pixels))
            else:
                means.append(0.0)
                stds.append(1.0)
        
        self.norm_means = np.array(means)
        self.norm_stds = np.array(stds)
        
        print(f"Calculated normalization parameters from combined datasets:")
        print(f"Band 1 (VV): mean={self.norm_means[0]:.4f}, std={self.norm_stds[0]:.4f}")
        print(f"Band 2 (NewBand1): mean={self.norm_means[1]:.4f}, std={self.norm_stds[1]:.4f}")
        print(f"Band 3 (NewBand2): mean={self.norm_means[2]:.4f}, std={self.norm_stds[2]:.4f}")
        
        # Save normalization parameters
        norm_params_path = os.path.join(self.output_path, 'normalization_params.npy')
        np.save(norm_params_path, {'means': self.norm_means, 'stds': self.norm_stds})
        print(f"Normalization parameters saved to: {norm_params_path}")

    def process_image(self, im_path):
        """Process image with normalization (second pass)"""
        try:
            with rasterio.open(im_path) as src:
                if src.count < 2:
                    print(f"Warning: Image {im_path} has only {src.count} band(s). Expected at least 2 (VH, VV)")
                    return None, None
                
                # Read VH and VV bands
                vh = src.read(1)
                vv = src.read(2)
                
                # Handle NaN and infinite values
                vh = np.nan_to_num(vh)
                vv = np.nan_to_num(vv)
                
                # Compute the three bands
                band1, band2, band3 = self.compute_bands(vh, vv)
                
                # Create 3-channel image
                arr_x = np.stack([band1, band2, band3], axis=0)
                
                # Clip extreme values
                for i in range(3):
                    v_min, v_max = np.percentile(arr_x[i], [1, 99])
                    arr_x[i] = np.clip(arr_x[i], v_min, v_max)
                
                # Resize to target dimensions
                arr_x = np.stack([
                    resize(arr_x[0], (self.img_size, self.img_size), preserve_range=True),
                    resize(arr_x[1], (self.img_size, self.img_size), preserve_range=True),
                    resize(arr_x[2], (self.img_size, self.img_size), preserve_range=True)
                ], axis=0)
                
                # Normalize using calculated means and stds
                if self.norm_means is not None and self.norm_stds is not None:
                    arr_x = (arr_x - self.norm_means.reshape(3, 1, 1)) / self.norm_stds.reshape(3, 1, 1)
                
                # Convert to HWC format for saving as image
                arr_x = np.transpose(arr_x, (1, 2, 0))
                
                # Scale to 0-1 range for visualization
                eps = 1e-8
                arr_x_viz = (arr_x - arr_x.min()) / (arr_x.max() - arr_x.min() + eps)
                
                return arr_x, arr_x_viz
                
        except Exception as e:
            print(f"Error processing image {im_path}: {str(e)}")
            return None, None

    def process_mask(self, dataset_type, mask_path):
        """
        Process mask from both dataset types with unified handling
        Returns: (ground_truth_mask, validity_mask)
        """
        try:
            with rasterio.open(mask_path) as src:
                arr_y = src.read(1)
            
            # Resize to target dimensions using nearest neighbor to preserve labels
            arr_y = resize(arr_y, (self.img_size, self.img_size), order=0, preserve_range=True)
            
            if dataset_type == 'otsu':
                # Otsu dataset: binary masks (0/1), all pixels are valid
                ground_truth_mask = (arr_y > 0).astype(np.uint8)
                validity_mask = np.ones_like(ground_truth_mask, dtype=np.uint8)  # All pixels valid
                
            elif dataset_type == 'sen1floods':
                # Sen1Floods11 dataset: has invalid pixels (-1)
                validity_mask = (arr_y != -1).astype(np.uint8)
                ground_truth_mask = arr_y.copy()
                ground_truth_mask[arr_y == -1] = 0  # Convert -1 to 0 for network compatibility
                ground_truth_mask = (ground_truth_mask > 0).astype(np.uint8)
            
            else:
                raise ValueError(f"Unknown dataset type: {dataset_type}")
            
            return ground_truth_mask, validity_mask
            
        except Exception as e:
            print(f"Error processing {dataset_type} mask {mask_path}: {str(e)}")
            return None, None

    def save_png(self, arr, save_path, mode='RGB'):
        """Save array as PNG image"""
        img = Image.fromarray((arr * 255).astype(np.uint8), mode=mode)
        img.save(save_path)

    def save_npy(self, arr, save_path):
        """Save raw array data as NPY file for preserving exact values"""
        np.save(save_path, arr)

    def update_stats(self, split, dataset_type, ground_truth_mask, validity_mask):
        """Update dataset statistics"""
        valid_pixels = validity_mask.astype(bool)
        
        self.stats[split]['count'] += 1
        self.stats[split]['flood_pixels'] += ground_truth_mask[valid_pixels].sum()
        self.stats[split]['total_valid_pixels'] += valid_pixels.sum()
        self.stats[split]['invalid_pixels'] += (~valid_pixels).sum()
        
        if dataset_type == 'otsu':
            self.stats[split]['otsu_count'] += 1
        elif dataset_type == 'sen1floods':
            self.stats[split]['sen1floods_count'] += 1

    def process_unified_dataset(self, split_name, pairs):
        """Process a unified dataset split"""
        print(f"Processing unified {split_name} dataset...")
        output_dir = os.path.join(self.output_path, split_name)
        
        successful_count = 0
        
        for dataset_type, base_id, im_path, mask_path in tqdm(pairs, desc=f"Processing {split_name}"):
            
            if not os.path.exists(im_path) or not os.path.exists(mask_path):
                print(f"Warning: Files not found - {im_path} or {mask_path}")
                continue
            
            # Process image (get both normalized data and visualization)
            arr_x, arr_x_viz = self.process_image(im_path)
            if arr_x is None:
                print(f"Failed to process image: {im_path}")
                continue
            
            # Process mask with dataset-specific handling
            result = self.process_mask(dataset_type, mask_path)
            if result is None:
                print(f"Failed to process mask: {mask_path}")
                continue
            
            ground_truth_mask, validity_mask = result
            
            # Save visualization as PNG
            img_save_path = os.path.join(output_dir, 'images', f'{split_name}_{successful_count:04d}.png')
            self.save_png(arr_x_viz, img_save_path, mode='RGB')
            
            # Save raw normalized data for exact values
            raw_save_path = os.path.join(output_dir, 'images', f'{split_name}_{successful_count:04d}.npy')
            self.save_npy(arr_x, raw_save_path)
            
            # Save ground truth mask as PNG and NPY
            mask_save_path = os.path.join(output_dir, 'masks', f'{split_name}_{successful_count:04d}.png')
            self.save_png(ground_truth_mask, mask_save_path, mode='L')
            mask_npy_path = os.path.join(output_dir, 'masks', f'{split_name}_{successful_count:04d}.npy')
            self.save_npy(ground_truth_mask, mask_npy_path)
            
            # Save validity mask as PNG and NPY
            validity_save_path = os.path.join(output_dir, 'validity_masks', f'{split_name}_{successful_count:04d}.png')
            self.save_png(validity_mask, validity_save_path, mode='L')
            validity_npy_path = os.path.join(output_dir, 'validity_masks', f'{split_name}_{successful_count:04d}.npy')
            self.save_npy(validity_mask, validity_npy_path)
            
            # Update statistics
            self.update_stats(split_name, dataset_type, ground_truth_mask, validity_mask)
            
            successful_count += 1
        
        print(f"Successfully processed {successful_count} samples for {split_name}")

    def print_stats(self):
        """Print comprehensive dataset statistics"""
        print("\nUnified Dataset Statistics (Valid Pixels Only):")
        print("=" * 80)
        total_samples = sum(stat['count'] for stat in self.stats.values())
        for split, stat in self.stats.items():
            if stat['count'] > 0:
                flood_percentage = 100 * stat['flood_pixels'] / stat['total_valid_pixels'] if stat['total_valid_pixels'] > 0 else 0
                invalid_percentage = 100 * stat['invalid_pixels'] / (stat['total_valid_pixels'] + stat['invalid_pixels'])
                split_percentage = 100 * stat['count'] / total_samples
                
                print(f"{split.upper()} set: {stat['count']} samples ({split_percentage:.1f}%)")
                print(f"  Dataset composition:")
                print(f"    Otsu: {stat['otsu_count']} samples")
                print(f"    Sen1Floods11: {stat['sen1floods_count']} samples")
                print(f"  Pixel statistics:")
                print(f"    Valid pixels: {stat['total_valid_pixels']:,} ({100-invalid_percentage:.1f}%)")
                print(f"    Invalid pixels: {stat['invalid_pixels']:,} ({invalid_percentage:.1f}%)")
                print(f"    Flood pixels (of valid): {stat['flood_pixels']:,} ({flood_percentage:.2f}%)")
                print()
        print("=" * 80)

    def calculate_class_weights(self):
        """Calculate class weights to handle imbalance (only for valid pixels)"""
        if self.stats['train']['total_valid_pixels'] > 0:
            pos_ratio = self.stats['train']['flood_pixels'] / self.stats['train']['total_valid_pixels']
            neg_ratio = 1 - pos_ratio
            
            # Class weights inversely proportional to class frequency
            weight_non_flood = 1.0
            weight_flood = neg_ratio / pos_ratio if pos_ratio > 0 else 1.0
            
            print(f"\nClass weights for handling imbalance (valid pixels only):")
            print(f"Weight for non-flood (0): {weight_non_flood:.4f}")
            print(f"Weight for flood (1): {weight_flood:.4f}")
            
            return np.array([weight_non_flood, weight_flood])
        return np.array([1.0, 1.0])

    def save_unified_splits_info(self, splits):
        """Save information about the unified splits for reproducibility"""
        splits_info = {}
        dataset_info = {
            'otsu_path': self.otsu_path,
            'sen1floods_path': self.sen1floods_path,
            'total_otsu_samples': 0,
            'total_sen1floods_samples': 0
        }
        
        for split_name, pairs in splits.items():
            splits_info[split_name] = []
            for dataset_type, base_id, im_path, mask_path in pairs:
                splits_info[split_name].append({
                    'dataset_type': dataset_type,
                    'base_id': base_id,
                    'image_path': im_path,
                    'mask_path': mask_path
                })
                
                if dataset_type == 'otsu':
                    dataset_info['total_otsu_samples'] += 1
                elif dataset_type == 'sen1floods':
                    dataset_info['total_sen1floods_samples'] += 1
        
        unified_info = {
            'dataset_info': dataset_info,
            'split_config': {
                'test_size': self.test_size,
                'val_size': self.val_size,
                'random_seed': self.random_seed
            },
            'splits': splits_info
        }
        
        with open(os.path.join(self.output_path, 'unified_splits_info.json'), 'w') as f:
            json.dump(unified_info, f, indent=2)
        
        print(f"Unified splits information saved to: {os.path.join(self.output_path, 'unified_splits_info.json')}")

# ==================== MAIN EXECUTION ====================

if __name__ == "__main__":
    print("Setting up Unified SAR Flood Dataset Preprocessing...")
    print("Combining S1F11-Otsu + Sen1Floods11 datasets")
    print("\nBand Configuration:")
    print("Band 1: VV")
    print("Band 2: NewBand1 = (VH - VV) / (VH + VV)")
    print("Band 3: NewBand2 = sqrt((VH² + VV²) / 2)")


    # Paths - UPDATE THESE TO YOUR ACTUAL PATHS
    otsu_path = "/kaggle/input/s1f11-otsu"  # Update this path
    sen1floods_path = "/kaggle/input/sen1floods11-essentials/v1.2"  # Update this path
    output_path = "/kaggle/working/unified_preprocessed"

    # Initialize unified preprocessor
    preprocessor = UnifiedSARFloodPreprocessor(
        otsu_path=otsu_path,
        sen1floods_path=sen1floods_path,
        output_path=output_path,
        test_size=0.2,  # 20% for test
        val_size=0.2,   # 20% for validation
        random_seed=42
    )

    # Step 1: Get all dataset pairs
    all_pairs = preprocessor.get_all_dataset_pairs()
    
    if len(all_pairs) == 0:
        print("ERROR: No dataset pairs found! Please check the dataset paths.")
        exit(1)

    # Step 2: Create unified splits
    splits = preprocessor.create_unified_splits(all_pairs)
    
    # Step 3: Save splits information for reproducibility
    preprocessor.save_unified_splits_info(splits)

    # Step 4: Calculate normalization parameters from combined training data
    preprocessor.calculate_normalization_params(splits['train'])

    print("\nStarting unified dataset preprocessing with calculated normalization parameters...")

    # Step 5: Process all unified datasets
    # Process train set
    preprocessor.process_unified_dataset('train', splits['train'])

    # Process validation set
    preprocessor.process_unified_dataset('val', splits['val'])

    # Process test set
    preprocessor.process_unified_dataset('test', splits['test'])

    # Print comprehensive statistics and calculate class weights
    preprocessor.print_stats()
    weights = preprocessor.calculate_class_weights()

    # Save class weights for later use
    weights_path = os.path.join(output_path, 'class_weights.npy')
    np.save(weights_path, weights)
    print(f"\nClass weights saved to: {weights_path}")

    print(f"\nUnified preprocessing complete! Processed data saved to: {output_path}")
    print("\nOutput structure:")
    print("unified_preprocessed/")
    print("├── train/")
    print("│   ├── images/ (PNG and NPY files)")
    print("│   ├── masks/ (PNG and NPY files - ground truth)")
    print("│   └── validity_masks/ (PNG and NPY files - valid pixel indicators)")
    print("├── val/")
    print("│   ├── images/ (PNG and NPY files)")
    print("│   ├── masks/ (PNG and NPY files - ground truth)")
    print("│   └── validity_masks/ (PNG and NPY files - valid pixel indicators)")
    print("├── test/")
    print("│   ├── images/ (PNG and NPY files)")
    print("│   ├── masks/ (PNG and NPY files - ground truth)")
    print("│   └── validity_masks/ (PNG and NPY files - valid pixel indicators)")
    print("├── class_weights.npy")
    print("├── normalization_params.npy")
    print("└── unified_splits_info.json")


## ResUnet Training Pipeline

In [ ]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import random
import glob
from tensorflow.keras import backend as K
import json
import pandas as pd
from PIL import Image
from skimage.transform import resize
import math

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

print("TensorFlow version:", tf.__version__)
print("GPU Available:", len(tf.config.list_physical_devices('GPU')) > 0)
print("GPU Devices:", tf.config.list_physical_devices('GPU'))

# Define paths
BASE_PATH = "/kaggle/working/unified_preprocessed"  # Updated to use preprocessed data
OUTPUT_PATH = "/kaggle/working/united_SEN1FLOODS11"
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Define helper functions for data loading with validity mask support
def load_image(image_path):
    """Load normalized image data from .npy file"""
    # Convert tensor to string if needed
    if isinstance(image_path, tf.Tensor):
        image_path = image_path.numpy().decode('utf-8')
    # Explicitly convert to float32
    return np.load(image_path).astype(np.float32)

def load_mask(mask_path):
    """Load mask from NPY file (preserves exact values)"""
    # Convert tensor to string if needed
    if isinstance(mask_path, tf.Tensor):
        mask_path = mask_path.numpy().decode('utf-8')
    mask = np.load(mask_path)
    # Explicitly convert to float32
    return mask.astype(np.float32)

def load_validity_mask(validity_path):
    """Load validity mask from NPY file"""
    # Convert tensor to string if needed
    if isinstance(validity_path, tf.Tensor):
        validity_path = validity_path.numpy().decode('utf-8')
    validity_mask = np.load(validity_path)
    # Explicitly convert to float32
    return validity_mask.astype(np.float32)

def augment_data(image, mask, validity_mask):
    """Apply data augmentation to image, mask, and validity mask"""
    # Random horizontal flip
    if tf.random.uniform([]) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
        validity_mask = tf.image.flip_left_right(validity_mask)
    
    # Random vertical flip
    if tf.random.uniform([]) > 0.5:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)
        validity_mask = tf.image.flip_up_down(validity_mask)
    
    # Random rotation (90, 180, 270 degrees)
    if tf.random.uniform([]) > 0.5:
        k = tf.random.uniform([], minval=1, maxval=4, dtype=tf.int32)
        image = tf.image.rot90(image, k=k)
        mask = tf.image.rot90(mask, k=k)
        validity_mask = tf.image.rot90(validity_mask, k=k)
    
    # Random brightness adjustment (only for image)
    if tf.random.uniform([]) > 0.5:
        image = tf.image.random_brightness(image, max_delta=0.1)
    
    # Random contrast adjustment (only for image)
    if tf.random.uniform([]) > 0.5:
        image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    
    return image, mask, validity_mask

def create_dataset(base_path, split, batch_size=16, shuffle=True, augment=False):
    """Create a TensorFlow dataset for the specified split with validity masks"""
    img_paths = sorted(glob.glob(os.path.join(base_path, split, 'images', '*.npy')))
    mask_paths = sorted(glob.glob(os.path.join(base_path, split, 'masks', '*.npy')))
    validity_paths = sorted(glob.glob(os.path.join(base_path, split, 'validity_masks', '*.npy')))

    if len(img_paths) == 0 or len(mask_paths) == 0 or len(validity_paths) == 0:
        raise ValueError(f"No images, masks, or validity masks found in {base_path}/{split}")

    print(f"Found {len(img_paths)} images, {len(mask_paths)} masks, and {len(validity_paths)} validity masks for {split}")

    # Create datasets of paths
    img_dataset = tf.data.Dataset.from_tensor_slices(img_paths)
    mask_dataset = tf.data.Dataset.from_tensor_slices(mask_paths)
    validity_dataset = tf.data.Dataset.from_tensor_slices(validity_paths)

    # Combine all paths
    dataset = tf.data.Dataset.zip((img_dataset, mask_dataset, validity_dataset))

    # Shuffle if needed
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(img_paths), seed=42)

    # Map loading function to the dataset
    dataset = dataset.map(
        lambda img_path, mask_path, validity_path: (
            tf.py_function(
                func=load_image,
                inp=[img_path],
                Tout=tf.float32
            ),
            tf.py_function(
                func=load_mask,
                inp=[mask_path],
                Tout=tf.float32
            ),
            tf.py_function(
                func=load_validity_mask,
                inp=[validity_path],
                Tout=tf.float32
            )
        ),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Set shapes
    dataset = dataset.map(
        lambda x, y, v: (
            tf.ensure_shape(x, [256, 256, 3]),
            tf.ensure_shape(y, [256, 256]),
            tf.ensure_shape(v, [256, 256])
        )
    )

    # Add channel dimension to masks
    dataset = dataset.map(lambda x, y, v: (x, tf.expand_dims(y, axis=-1), tf.expand_dims(v, axis=-1)))

    # Apply data augmentation for training set
    if augment and split == 'train':
        print(f"Applying data augmentation to {split} dataset")
        dataset = dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

    # Important: Add repeat to prevent dataset exhaustion
    dataset = dataset.repeat()
    
    # Batch and prefetch
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset, len(img_paths)

def visualize_samples(dataset, num_samples=3):
    """Visualize random samples from the dataset with validity masks"""
    plt.figure(figsize=(20, 5*num_samples))

    for i, (images, masks, validity_masks) in enumerate(dataset.take(num_samples)):
        for j in range(min(images.shape[0], 3)):
            # Get image, mask, and validity mask
            image = images[j].numpy()
            mask = masks[j].numpy().squeeze()
            validity_mask = validity_masks[j].numpy().squeeze()

            # Normalize image for visualization (0-1 range)
            image_viz = (image - image.min()) / (image.max() - image.min() + 1e-8)

            # Display RGB channels
            plt.subplot(num_samples, 5, i*5+1)
            plt.imshow(image_viz[:, :, 0], cmap='gray')
            plt.title(f"VV Channel - Sample {i+1}")
            plt.axis('off')

            plt.subplot(num_samples, 5, i*5+2)
            plt.imshow(image_viz[:, :, 1], cmap='gray')
            plt.title(f"NewBand1 - Sample {i+1}")
            plt.axis('off')

            plt.subplot(num_samples, 5, i*5+3)
            plt.imshow(image_viz[:, :, 2], cmap='gray')
            plt.title(f"NewBand2 - Sample {i+1}")
            plt.axis('off')

            # Display ground truth mask
            plt.subplot(num_samples, 5, i*5+4)
            plt.imshow(mask, cmap='Blues')
            plt.title(f"Ground Truth - Sample {i+1}")
            plt.axis('off')

            # Display validity mask
            plt.subplot(num_samples, 5, i*5+5)
            plt.imshow(validity_mask, cmap='Reds')
            plt.title(f"Validity Mask - Sample {i+1}")
            plt.axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, 'sample_visualization_with_validity.png'))
    plt.show()

# Define MASKED custom metrics and loss functions that ignore invalid pixels
def masked_dice_coefficient(y_true, y_pred, validity_mask, smooth=1e-6):
    """Calculate Dice coefficient only for valid pixels"""
    # Apply validity mask
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)
    
    # Only consider valid pixels
    intersection = K.sum(y_true_f * y_pred_f * validity_f)
    union = K.sum(y_true_f * validity_f) + K.sum(y_pred_f * validity_f)
    
    return (2. * intersection + smooth) / (union + smooth)

def masked_dice_loss(y_true_and_validity, y_pred):
    """Masked Dice loss function that ignores invalid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    return 1 - masked_dice_coefficient(y_true, y_pred, validity_mask)

def masked_focal_tversky_loss(y_true_and_validity, y_pred, alpha=0.7, beta=0.3, gamma=1.5, smooth=1e-6):
    """
    Masked Focal Tversky Loss that ignores invalid pixels
    """
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    # Apply validity mask
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    # Flatten the inputs
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)
    
    # Calculate true positives, false negatives, and false positives (only for valid pixels)
    true_pos = K.sum(y_true_f * y_pred_f * validity_f)
    false_neg = K.sum(y_true_f * (1 - y_pred_f) * validity_f)
    false_pos = K.sum((1 - y_true_f) * y_pred_f * validity_f)
    
    # Calculate Tversky index
    tversky = (true_pos + smooth) / (true_pos + alpha * false_neg + beta * false_pos + smooth)
    
    # Apply focal parameter to focus on hard examples
    focal_tversky = K.pow((1 - tversky), gamma)
    
    return focal_tversky

def masked_iou_score(y_true_and_validity, y_pred, smooth=1e-6):
    """Calculate IoU score only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    # Apply validity mask
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)
    
    intersection = K.sum(y_true_f * y_pred_f * validity_f)
    union = K.sum(y_true_f * validity_f) + K.sum(y_pred_f * validity_f) - intersection
    
    return (intersection + smooth) / (union + smooth)

def masked_binary_accuracy(y_true_and_validity, y_pred):
    """Calculate binary accuracy only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    # Apply validity mask
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    # Threshold predictions
    y_pred_binary = K.cast(y_pred_masked > 0.5, K.floatx())
    
    # Calculate accuracy only for valid pixels
    correct = K.cast(K.equal(y_true_masked, y_pred_binary), K.floatx()) * validity_mask
    total_valid = K.sum(validity_mask)
    
    return K.sum(correct) / (total_valid + K.epsilon())

def masked_f1_score_metric(y_true_and_validity, y_pred, smooth=1e-6):
    """Calculate F1 score only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    # Apply validity mask
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)

    # Calculate precision and recall for valid pixels only
    true_positives = K.sum(y_true_f * y_pred_f * validity_f)
    predicted_positives = K.sum(y_pred_f * validity_f)
    actual_positives = K.sum(y_true_f * validity_f)

    precision = (true_positives + smooth) / (predicted_positives + smooth)
    recall = (true_positives + smooth) / (actual_positives + smooth)

    # Calculate F1 score
    f1 = 2 * (precision * recall) / (precision + recall + smooth)
    return f1

def masked_precision_metric(y_true_and_validity, y_pred, smooth=1e-6):
    """Calculate precision only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    # Apply validity mask
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)

    true_positives = K.sum(y_true_f * y_pred_f * validity_f)
    predicted_positives = K.sum(y_pred_f * validity_f)

    precision = (true_positives + smooth) / (predicted_positives + smooth)
    return precision

def masked_recall_metric(y_true_and_validity, y_pred, smooth=1e-6):
    """Calculate recall only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    # Apply validity mask
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)

    true_positives = K.sum(y_true_f * y_pred_f * validity_f)
    actual_positives = K.sum(y_true_f * validity_f)

    recall = (true_positives + smooth) / (actual_positives + smooth)
    return recall

# Wrapper functions to use with compile (need specific names)
def masked_dice_coefficient_metric(y_true_and_validity, y_pred):
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    return masked_dice_coefficient(y_true, y_pred, validity_mask)

# Define ResUNet model architecture (unchanged)
def conv_block(inputs, filters, kernel_size=3, strides=1, padding='same'):
    """Convolutional block with batch normalization and activation"""
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding=padding)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def channel_attention(inputs, ratio=16):
    """Squeeze and Excitation Block for channel attention"""
    channels = int(inputs.shape[-1])
    reduced_channels = max(channels // ratio, 8)
    
    x = layers.GlobalAveragePooling2D()(inputs)
    x = layers.Reshape((1, 1, channels))(x)
    x = layers.Conv2D(reduced_channels, kernel_size=1, activation='relu', padding='same')(x)
    x = layers.Conv2D(channels, kernel_size=1, activation='sigmoid', padding='same')(x)
    
    output = layers.Multiply()([inputs, x])
    return output

def spatial_attention(inputs):
    """Spatial attention module"""
    avg_pool = layers.Conv2D(1, kernel_size=1, padding='same', use_bias=False, 
                            kernel_initializer='ones')(inputs)
    max_features = layers.Conv2D(1, kernel_size=7, padding='same', activation='relu')(inputs)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_features])
    attention_map = layers.Conv2D(1, kernel_size=7, padding='same', activation='sigmoid')(concat)
    output = layers.Multiply()([inputs, attention_map])
    return output

def attention_residual_block(inputs, filters, kernel_size=3, strides=1):
    """Residual block with channel and spatial attention"""
    x = conv_block(inputs, filters, kernel_size, strides)
    x = conv_block(x, filters, kernel_size, 1)
    x = channel_attention(x)
    x = spatial_attention(x)
    
    if strides > 1 or inputs.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')(inputs)
        shortcut = layers.BatchNormalization()(shortcut)
    else:
        shortcut = inputs
    
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    
    return x

def build_attention_resunet(input_shape=(256, 256, 3), num_classes=1):
    """Build ResUNet model with attention mechanisms"""
    inputs = layers.Input(input_shape)

    x = conv_block(inputs, 64, kernel_size=7, strides=1)

    # Encoder blocks
    skip1 = attention_residual_block(x, 64)
    x = layers.MaxPooling2D(2)(skip1)

    skip2 = attention_residual_block(x, 128)
    x = layers.MaxPooling2D(2)(skip2)

    skip3 = attention_residual_block(x, 256)
    x = layers.MaxPooling2D(2)(skip3)

    # Bridge
    x = attention_residual_block(x, 512)

    # Decoder blocks
    x = layers.UpSampling2D(2)(x)
    x = conv_block(x, 256)
    x = layers.Concatenate()([x, skip3])
    x = attention_residual_block(x, 256)

    x = layers.UpSampling2D(2)(x)
    x = conv_block(x, 128)
    x = layers.Concatenate()([x, skip2])
    x = attention_residual_block(x, 128)

    x = layers.UpSampling2D(2)(x)
    x = conv_block(x, 64)
    x = layers.Concatenate()([x, skip1])
    x = attention_residual_block(x, 64)

    # Output layer
    outputs = layers.Conv2D(num_classes, kernel_size=1, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Cosine Annealing Learning Rate Classes (unchanged from original)
class CosineAnnealingCallback(callbacks.Callback):
    """Custom Cosine Annealing Learning Rate Callback with visualization"""
    
    def __init__(self, initial_lr, min_lr, T_max, verbose=1):
        super(CosineAnnealingCallback, self).__init__()
        self.initial_lr = initial_lr
        self.min_lr = min_lr
        self.T_max = T_max
        self.verbose = verbose
        self.learning_rates = []
        
    def on_epoch_begin(self, epoch, logs=None):
        lr = self.min_lr + (self.initial_lr - self.min_lr) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        
        if hasattr(self.model.optimizer, 'learning_rate'):
            if hasattr(self.model.optimizer.learning_rate, 'assign'):
                self.model.optimizer.learning_rate.assign(lr)
            else:
                self.model.optimizer.learning_rate = lr
        else:
            self.model.optimizer.lr = lr
            
        self.learning_rates.append(lr)
        
        if self.verbose > 0:
            print(f"Epoch {epoch + 1}: Learning rate = {lr:.6f}")
    
    def on_train_end(self, logs=None):
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(self.learning_rates) + 1), self.learning_rates)
        plt.title('Cosine Annealing Learning Rate Schedule')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(OUTPUT_PATH, 'cosine_annealing_lr_schedule.png'))
        plt.show()

def cosine_annealing_warm_restarts(epoch, initial_lr=0.001, min_lr=1e-6, T_0=50, T_mult=2):
    """Cosine annealing with warm restarts function for LearningRateScheduler"""
    T_cur = epoch
    T_i = T_0
    
    while T_cur >= T_i:
        T_cur -= T_i
        T_i *= T_mult
    
    lr = min_lr + (initial_lr - min_lr) * (1 + math.cos(math.pi * T_cur / T_i)) / 2
    return lr

def plot_history(history, lr_schedule_type="cosine_annealing"):
    """Plot comprehensive training history with all metrics including learning rate"""
    plt.figure(figsize=(20, 18))

    # Plot loss
    plt.subplot(3, 3, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Masked Loss (Invalid Pixels Excluded)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    # Plot Dice coefficient
    plt.subplot(3, 3, 2)
    plt.plot(history.history['masked_dice_coefficient_metric'], label='Training Dice')
    plt.plot(history.history['val_masked_dice_coefficient_metric'], label='Validation Dice')
    plt.title('Masked Dice Coefficient')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    # Plot IoU
    plt.subplot(3, 3, 3)
    plt.plot(history.history['masked_iou_score'], label='Training IoU')
    plt.plot(history.history['val_masked_iou_score'], label='Validation IoU')
    plt.title('Masked IoU Score')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    # Plot binary accuracy
    plt.subplot(3, 3, 4)
    plt.plot(history.history['masked_binary_accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_masked_binary_accuracy'], label='Validation Accuracy')
    plt.title('Masked Binary Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    # Plot F1 Score
    plt.subplot(3, 3, 5)
    plt.plot(history.history['masked_f1_score_metric'], label='Training F1')
    plt.plot(history.history['val_masked_f1_score_metric'], label='Validation F1')
    plt.title('Masked F1 Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    # Plot Precision and Recall
    plt.subplot(3, 3, 6)
    plt.plot(history.history['masked_precision_metric'], label='Training Precision')
    plt.plot(history.history['val_masked_precision_metric'], label='Validation Precision')
    plt.plot(history.history['masked_recall_metric'], label='Training Recall')
    plt.plot(history.history['val_masked_recall_metric'], label='Validation Recall')
    plt.title('Masked Precision and Recall')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)

    # Plot Learning Rate
    plt.subplot(3, 3, 7)
    if 'lr' in history.history:
        plt.plot(history.history['lr'], label='Learning Rate', color='red')
        plt.title(f'Learning Rate Schedule ({lr_schedule_type})')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.yscale('log')
    else:
        plt.text(0.5, 0.5, 'Learning Rate\nNot Available', 
                horizontalalignment='center', verticalalignment='center', 
                transform=plt.gca().transAxes, fontsize=12)
        plt.title('Learning Rate Schedule')

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, f'training_history_{lr_schedule_type}_masked.png'), dpi=300, bbox_inches='tight')
    plt.show()

def visualize_predictions(model, dataset, num_samples=5):
    """Visualize model predictions with complete validity mask information"""
    plt.figure(figsize=(30, 6*num_samples))

    sample_count = 0
    for images, combined_masks in dataset:
        if sample_count >= num_samples:
            break

        # Get predictions
        preds = model.predict(images, verbose=0)

        for j in range(min(images.shape[0], num_samples - sample_count)):
            # Get image, masks, and prediction
            image = images[j].numpy()
            gt_mask = combined_masks[j, ..., 0].numpy()  # Ground truth
            validity_mask = combined_masks[j, ..., 1].numpy()  # Validity mask
            pred = preds[j].squeeze()

            # Normalize image for visualization
            image_viz = (image - image.min()) / (image.max() - image.min() + 1e-8)

            # Create RGB composite
            rgb_viz = np.stack([
                image_viz[:, :, 0],  # VV as R
                image_viz[:, :, 1],  # NewBand1 as G
                image_viz[:, :, 2]   # NewBand2 as B
            ], axis=-1)
            rgb_viz = np.clip(rgb_viz, 0, 1)

            # Calculate masked metrics for this sample (only valid pixels)
            valid_pixels = validity_mask > 0.5
            invalid_pixels = validity_mask <= 0.5
            
            if np.sum(valid_pixels) > 0:
                gt_valid = gt_mask[valid_pixels]
                pred_valid = pred[valid_pixels]
                pred_binary_valid = (pred_valid > 0.5).astype(np.float32)
                gt_binary_valid = (gt_valid > 0.5).astype(np.float32)
                
                dice = np.sum(2 * gt_binary_valid * pred_binary_valid) / (np.sum(gt_binary_valid) + np.sum(pred_binary_valid) + 1e-8)
                intersection = np.sum(gt_binary_valid * pred_binary_valid)
                union = np.sum(gt_binary_valid) + np.sum(pred_binary_valid) - intersection
                iou = intersection / (union + 1e-8)
            else:
                dice = iou = 0.0

            valid_percentage = 100 * np.sum(valid_pixels) / valid_pixels.size
            invalid_percentage = 100 * np.sum(invalid_pixels) / invalid_pixels.size

            row = sample_count
            
            # 1. Original SAR composite
            plt.subplot(num_samples, 6, row * 6 + 1)
            plt.imshow(rgb_viz)
            plt.title(f'Sample {sample_count+1}\nSAR Composite\n(VV, NewBand1, NewBand2)')
            plt.axis('off')

            # 2. Ground truth with invalid areas grayed out
            plt.subplot(num_samples, 6, row * 6 + 2)
            gt_display = np.ones_like(gt_mask) * 0.5  # Gray for invalid areas
            gt_display[valid_pixels] = gt_mask[valid_pixels]  # Valid GT values
            plt.imshow(gt_display, cmap='Blues', vmin=0, vmax=1)
            plt.title(f'Ground Truth\n(Gray = Invalid)\nValid: {valid_percentage:.1f}%')
            plt.axis('off')

            # 3. Validity mask
            plt.subplot(num_samples, 6, row * 6 + 3)
            plt.imshow(validity_mask, cmap='RdYlGn', vmin=0, vmax=1)
            plt.title(f'Validity Mask\n(Red = Invalid: {invalid_percentage:.1f}%)\n(Green = Valid: {valid_percentage:.1f}%)')
            plt.axis('off')

            # 4. Model prediction (all pixels)
            plt.subplot(num_samples, 6, row * 6 + 4)
            plt.imshow(pred, cmap='Blues', vmin=0, vmax=1)
            plt.title(f'Model Prediction\n(All pixels)')
            plt.axis('off')

            # 5. Masked prediction (only valid areas shown)
            plt.subplot(num_samples, 6, row * 6 + 5)
            pred_masked = pred.copy()
            pred_masked[invalid_pixels] = 0.5  # Gray out invalid areas
            plt.imshow(pred_masked, cmap='Blues', vmin=0, vmax=1)
            plt.title(f'Masked Prediction\n(Only valid areas)')
            plt.axis('off')

            # 6. Error map (only for valid pixels)
            plt.subplot(num_samples, 6, row * 6 + 6)
            error = np.abs(gt_mask - pred) * validity_mask
            error[invalid_pixels] = 0  # No error contribution from invalid pixels
            plt.imshow(error, cmap='Reds', vmin=0, vmax=1)
            plt.title(f'Error Map (Valid Only)\nDice: {dice:.3f}, IoU: {iou:.3f}\nNo invalid pixel bias!')
            plt.axis('off')

            sample_count += 1
            if sample_count >= num_samples:
                break
        
        if sample_count >= num_samples:
            break

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_PATH, 'masked_predictions_with_validity.png'), dpi=300, bbox_inches='tight')
    plt.show()

# ==================== MAIN EXECUTION WITH MASKED LOSS ====================

# Configuration for Cosine Annealing
COSINE_ANNEALING_CONFIG = {
    "method": "warm_restarts_scheduler",
    "initial_lr": 0.0005,
    "min_lr": 5e-7,
    "T_max": 200,
    "T_0": 50,
    "T_mult": 2,
}

print(f"Using Cosine Annealing method: {COSINE_ANNEALING_CONFIG['method']}")
print("🔧 TRAINING WITH MASKED LOSS AND METRICS")
print("✅ Invalid ground truth pixels (-1) will be COMPLETELY EXCLUDED from:")
print("   • Loss calculation")
print("   • All metric calculations")
print("   • Model learning")

# Use a batch size that fits in GPU memory
BATCH_SIZE = 24

# Create datasets with validity masks
print("Creating datasets with validity mask support...")
train_dataset, train_size = create_dataset(BASE_PATH, 'train', batch_size=BATCH_SIZE, augment=True)
val_dataset, val_size = create_dataset(BASE_PATH, 'val', batch_size=BATCH_SIZE, augment=False)
test_dataset, test_size = create_dataset(BASE_PATH, 'test', batch_size=BATCH_SIZE, augment=False)

print(f"Training dataset size: {train_size} images")
print(f"Validation dataset size: {val_size} images")
print(f"Test dataset size: {test_size} images")

# Visualize some samples with validity masks
visualize_samples(train_dataset, num_samples=2)

# Build model
input_shape = (256, 256, 3)
model = build_attention_resunet(input_shape)

# Calculate steps per epoch
steps_per_epoch = max(1, train_size // BATCH_SIZE)
validation_steps = max(1, val_size // BATCH_SIZE)

print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")

# Setup cosine annealing
method = COSINE_ANNEALING_CONFIG["method"]
initial_lr = COSINE_ANNEALING_CONFIG["initial_lr"]

optimizer = optimizers.Adam(learning_rate=initial_lr)
lr_schedule_func = lambda epoch: cosine_annealing_warm_restarts(
    epoch,
    initial_lr=initial_lr,
    min_lr=COSINE_ANNEALING_CONFIG["min_lr"],
    T_0=COSINE_ANNEALING_CONFIG["T_0"],
    T_mult=COSINE_ANNEALING_CONFIG["T_mult"]
)
lr_callback = callbacks.LearningRateScheduler(lr_schedule_func, verbose=1)

print(f" Successfully configured cosine annealing method: {method}")

# Custom data preparation function to combine ground truth and validity masks
def prepare_labels_for_training(dataset):
    """Prepare dataset to combine ground truth and validity masks for masked loss"""
    def combine_masks(image, gt_mask, validity_mask):
        # Combine ground truth and validity masks into a single tensor
        combined_mask = tf.concat([gt_mask, validity_mask], axis=-1)
        return image, combined_mask
    
    return dataset.map(combine_masks)

# Prepare datasets for training with combined masks
print("Preparing datasets for masked training...")
train_dataset_prepared = prepare_labels_for_training(train_dataset)
val_dataset_prepared = prepare_labels_for_training(val_dataset)
test_dataset_prepared = prepare_labels_for_training(test_dataset)

# Compile model with masked loss and metrics
print("Compiling model with MASKED loss and metrics...")
model.compile(
    optimizer=optimizer,
    loss=masked_focal_tversky_loss,  # Masked loss that ignores invalid pixels
    metrics=[
        masked_dice_coefficient_metric,
        masked_iou_score,
        masked_binary_accuracy,
        masked_f1_score_metric,
        masked_precision_metric,
        masked_recall_metric
    ]
)

# Display model summary
print("Model Architecture:")
model.summary()

# Define callbacks
checkpoint_path = os.path.join(OUTPUT_PATH, "best_model_masked_cosine_annealing.keras")
log_dir = os.path.join(OUTPUT_PATH, "logs_masked_cosine_annealing")
os.makedirs(log_dir, exist_ok=True)

callbacks_list = [
    callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_masked_iou_score',
        save_best_only=True,
        mode='max'
    ),
    callbacks.EarlyStopping(
        monitor='val_masked_iou_score',
        patience=70,
        restore_best_weights=True,
        mode='max'
    ),
    callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        update_freq='epoch',
        write_graph=True,
        write_images=True,
        profile_batch=0
    ),
    callbacks.CSVLogger(
        os.path.join(OUTPUT_PATH, f'training_log_masked_cosine_{method}.csv'),
        separator=',',
        append=False
    ),
    lr_callback
]

print(f"Training with {method} cosine annealing and MASKED loss/metrics...")
print(f"Initial LR: {initial_lr}, Min LR: {COSINE_ANNEALING_CONFIG['min_lr']}")

# Train model with masked datasets
epochs = 50
history = model.fit(
    train_dataset_prepared,
    validation_data=val_dataset_prepared,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks_list
)

# Plot training history
plot_history(history, lr_schedule_type=f"masked_cosine_{method}")

# Load best model
best_model = models.load_model(checkpoint_path, custom_objects={
    'masked_dice_coefficient_metric': masked_dice_coefficient_metric,
    'masked_focal_tversky_loss': masked_focal_tversky_loss,
    'masked_iou_score': masked_iou_score,
    'masked_binary_accuracy': masked_binary_accuracy,
    'masked_f1_score_metric': masked_f1_score_metric,
    'masked_precision_metric': masked_precision_metric,
    'masked_recall_metric': masked_recall_metric
})

# Evaluate on test set
print("Evaluating on test set with MASKED metrics...")
test_steps = max(1, test_size // BATCH_SIZE)
test_results = best_model.evaluate(test_dataset_prepared, steps=test_steps)
print("\nTest Results (Invalid pixels excluded):")
for metric_name, value in zip(best_model.metrics_names, test_results):
    print(f"{metric_name}: {value:.4f}")

# Save test results
test_metrics = {metric_name: float(value) for metric_name, value in zip(best_model.metrics_names, test_results)}
test_metrics['cosine_annealing_method'] = method
test_metrics['cosine_annealing_config'] = COSINE_ANNEALING_CONFIG
test_metrics['masked_training'] = True

with open(os.path.join(OUTPUT_PATH, f'test_metrics_masked_cosine_{method}.json'), 'w') as f:
    json.dump(test_metrics, f, indent=4)
print("Test metrics saved to JSON file")

def training_consistent_masked_iou_score_eval(combined_masks, y_pred, threshold=0.5, smooth=1e-6):
    """
    Calculate IoU using the EXACT same method as during training for consistency
    """
    # Extract masks
    y_true = combined_masks[..., 0:1]  # Ground truth
    validity_mask = combined_masks[..., 1:2]  # Validity mask
    
    # Apply threshold to predictions (convert to binary)
    y_pred_binary = tf.cast(y_pred > threshold, tf.float32)
    y_true_binary = tf.cast(y_true > threshold, tf.float32)
    
    # Apply validity mask - COMPLETELY exclude invalid pixels
    y_true_masked = y_true_binary * validity_mask
    y_pred_masked = y_pred_binary * validity_mask
    
    # Flatten (same as training)
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)
    
    # Calculate intersection and union ONLY for valid pixels
    intersection = K.sum(y_true_f * y_pred_f * validity_f)
    union = K.sum(y_true_f * validity_f) + K.sum(y_pred_f * validity_f) - intersection
    
    iou = (intersection + smooth) / (union + smooth)
    return iou

def calculate_masked_metrics(model, dataset, num_batches=None, threshold=0.5):
    """
    FIXED: Calculate comprehensive metrics using TRAINING-CONSISTENT methods
    Ensures COMPLETE exclusion of invalid pixels (-1) from all calculations
    """
    print(f"\n COMPREHENSIVE MASKED EVALUATION (TRAINING-CONSISTENT)")
    print(f" Invalid pixels (-1) COMPLETELY EXCLUDED from ALL calculations")
    print("=" * 70)
    
    # Collect data using training-consistent methods
    training_consistent_ious = []  # Per-batch IoU (matches training exactly)
    all_y_true_valid = []
    all_y_pred_valid = []
    image_metrics = []
    total_valid_pixels = 0
    total_invalid_pixels = 0
    
    batch_count = 0
    
    for images, combined_masks in dataset:
        if num_batches is not None and batch_count >= num_batches:
            break
            
        batch_count += 1
        print(f"Processing batch {batch_count}...")
        
        # Get predictions
        preds = model.predict(images, verbose=0)
        
        # METHOD 1: Calculate IoU the SAME way as training (per-batch)
        batch_iou = training_consistent_masked_iou_score_eval(combined_masks, preds, threshold)
        training_consistent_ious.append(float(batch_iou))
        
        # METHOD 2: Process individual images for detailed metrics
        gt_masks = combined_masks[..., 0]  # Ground truth
        validity_masks = combined_masks[..., 1]  # Validity mask
        
        for i in range(len(images)):
            gt_mask = gt_masks[i].numpy()
            pred = preds[i].squeeze()
            validity_mask = validity_masks[i].numpy()
            
            # Count pixels
            valid_pixels_mask = validity_mask > 0.5
            invalid_pixels_mask = validity_mask <= 0.5
            
            num_valid = np.sum(valid_pixels_mask)
            num_invalid = np.sum(invalid_pixels_mask)
            
            total_valid_pixels += num_valid
            total_invalid_pixels += num_invalid
            
            if num_valid == 0:
                print(f"Warning: Image {i} has no valid pixels, skipping...")
                continue
            
            # Extract ONLY valid pixels
            gt_valid = gt_mask[valid_pixels_mask]
            pred_valid = pred[valid_pixels_mask]
            
            # Apply threshold
            pred_binary = (pred_valid > threshold).astype(np.int32)
            gt_binary = (gt_valid > threshold).astype(np.int32)
            
            # Add to overall collection
            all_y_true_valid.extend(gt_binary)
            all_y_pred_valid.extend(pred_binary)
            
            # Calculate per-image metrics
            intersection = np.sum(gt_binary * pred_binary)
            union = np.sum(gt_binary) + np.sum(pred_binary) - intersection
            
            img_iou = (intersection + 1e-8) / (union + 1e-8)
            img_dice = (2 * intersection + 1e-8) / (np.sum(gt_binary) + np.sum(pred_binary) + 1e-8)
            
            # Confusion matrix for valid pixels only
            try:
                cm = confusion_matrix(gt_binary, pred_binary, labels=[0, 1])
                if cm.shape == (2, 2):
                    tn, fp, fn, tp = cm.ravel()
                else:
                    tn = fp = fn = tp = 0
            except:
                tn = fp = fn = tp = 0
            
            # Per-image metrics
            img_precision = tp / (tp + fp + 1e-8)
            img_recall = tp / (tp + fn + 1e-8)
            img_f1 = 2 * (img_precision * img_recall) / (img_precision + img_recall + 1e-8)
            img_accuracy = (tp + tn) / (tp + tn + fp + fn + 1e-8)
            
            image_metrics.append({
                'iou': img_iou,
                'dice': img_dice,
                'precision': img_precision,
                'recall': img_recall,
                'f1': img_f1,
                'accuracy': img_accuracy,
                'tp': int(tp), 'fp': int(fp), 'tn': int(tn), 'fn': int(fn),
                'valid_pixels': int(num_valid),
                'invalid_pixels': int(num_invalid),
                'valid_percentage': 100 * num_valid / (num_valid + num_invalid)
            })
    
    if len(all_y_true_valid) == 0:
        print(" No valid pixels found!")
        return {}
    
    print(f"\n PIXEL STATISTICS:")
    print(f"Total valid pixels: {total_valid_pixels:,}")
    print(f"Total invalid pixels: {total_invalid_pixels:,}")
    total_pixels = total_valid_pixels + total_invalid_pixels
    print(f"Valid pixel percentage: {100 * total_valid_pixels / total_pixels:.2f}%")
    print(f"Invalid pixel percentage: {100 * total_invalid_pixels / total_pixels:.2f}%")
    
    # Calculate overall metrics from ALL valid pixels
    try:
        cm = confusion_matrix(all_y_true_valid, all_y_pred_valid, labels=[0, 1])
        if cm.shape == (2, 2):
            tn, fp, fn, tp = cm.ravel()
        else:
            print("Warning: Unusual confusion matrix shape")
            tp = fp = fn = tn = 0
    except Exception as e:
        print(f"Error in confusion matrix: {e}")
        tp = fp = fn = tn = 0
    
    # Calculate metrics
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    accuracy = (tp + tn) / (tp + tn + fp + fn + 1e-8)
    specificity = tn / (tn + fp + 1e-8)
    
    # IoU from confusion matrix
    intersection_cm = tp
    union_cm = tp + fp + fn
    iou_from_cm = intersection_cm / (union_cm + 1e-8)
    
    # TRAINING-CONSISTENT IoU (this should match your validation IoU during training)
    training_consistent_iou = np.mean(training_consistent_ious)
    
    # Per-image averages
    mean_dice = np.mean([m['dice'] for m in image_metrics]) if image_metrics else 0.0
    mean_iou_per_image = np.mean([m['iou'] for m in image_metrics]) if image_metrics else 0.0
    
    print(f"\n======== COMPLETE Masked Metrics (Valid Pixels Only) ========")
    print(f" TRAINING-CONSISTENT IoU: {training_consistent_iou:.4f} <- Should match validation IoU!")
    print(f"")
    print(f"Overall Metrics (pixel-level):")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall/Sensitivity: {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"IoU: {iou_from_cm:.4f}")
    print(f"Dice Coefficient: {(2 * tp) / (2 * tp + fp + fn + 1e-8):.4f}")
    
    print(f"\nMean Per-Image Metrics:")
    print(f"Mean Dice: {mean_dice:.4f}")
    print(f"Mean IoU: {mean_iou_per_image:.4f}")
    
    print(f"\nConfusion Matrix (Valid Pixels Only):")
    print(f"True Positives: {tp:,}")
    print(f"False Positives: {fp:,}")
    print(f"True Negatives: {tn:,}")
    print(f"False Negatives: {fn:,}")
    
    # Per-class metrics
    bg_precision = tn / (tn + fn + 1e-8)
    bg_recall = tn / (tn + fp + 1e-8)
    bg_f1 = 2 * (bg_precision * bg_recall) / (bg_precision + bg_recall + 1e-8)
    
    print(f"\nPer-Class Metrics:")
    print(f"Background - Precision: {bg_precision:.4f}, Recall: {bg_recall:.4f}, F1: {bg_f1:.4f}")
    print(f"Flood - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    
    # Save detailed results
    if image_metrics:
        img_metrics_df = pd.DataFrame(image_metrics)
        img_metrics_df.to_csv(os.path.join(OUTPUT_PATH, 'per_image_metrics_FIXED_COMPLETE.csv'), index_label='image_id')
        print(f"\n Per-image metrics saved to CSV file")
    
    return {
        'overall': {
            'accuracy': float(accuracy),
            'precision': float(precision),
            'recall': float(recall),
            'specificity': float(specificity),
            'f1': float(f1),
            'iou_training_consistent': float(training_consistent_iou),  # KEY METRIC!
            'iou_confusion_matrix': float(iou_from_cm),
            'dice': float((2 * tp) / (2 * tp + fp + fn + 1e-8))
        },
        'per_image_mean': {
            'dice': float(mean_dice),
            'iou': float(mean_iou_per_image)
        },
        'confusion_matrix': {
            'tn': int(tn), 'fp': int(fp), 'fn': int(fn), 'tp': int(tp)
        },
        'per_class': {
            'background': {'precision': float(bg_precision), 'recall': float(bg_recall), 'f1': float(bg_f1)},
            'flood': {'precision': float(precision), 'recall': float(recall), 'f1': float(f1)}
        },
        'pixel_statistics': {
            'total_valid_pixels': int(total_valid_pixels),
            'total_invalid_pixels': int(total_invalid_pixels),
            'valid_percentage': float(100 * total_valid_pixels / total_pixels),
            'invalid_percentage': float(100 * total_invalid_pixels / total_pixels)
        },
        'evaluation_info': {
            'num_images_evaluated': len(image_metrics),
            'num_batches_processed': len(training_consistent_ious),
            'threshold_used': threshold,
            'completely_masked': True
        }
    }

# Visualize predictions with validity masks
print("\n  VISUALIZING PREDICTIONS WITH VALIDITY MASKS...")
try:
    # Try with prepared dataset first
    visualize_predictions(best_model, test_dataset_prepared, num_samples=5)
except Exception as e:
    print(f"  Error with prepared dataset: {e}")
    print(" Trying with original dataset format...")
    try:
        visualize_predictions(best_model, test_dataset, num_samples=5)
    except Exception as e2:
        print(f" Error with original dataset: {e2}")
        print("Skipping visualization...")

# Calculate comprehensive masked metrics on test set (limit to a reasonable number of batches)
# Calculate comprehensive masked metrics on test set (limit to a reasonable number of batches)
test_batches_for_eval = min(10, test_size // BATCH_SIZE)
print(f"\nCalculating detailed masked metrics on {test_batches_for_eval} test batches...")
detailed_metrics = calculate_masked_metrics(best_model, test_dataset_prepared, num_batches=test_batches_for_eval)

# Add training configuration to detailed metrics
if detailed_metrics:
    detailed_metrics['training_config'] = {
        'masked_training': True,
        'cosine_annealing_method': method,
        'cosine_annealing_params': COSINE_ANNEALING_CONFIG,
        'epochs_trained': len(history.history['loss']),
        'batch_size': BATCH_SIZE,
        'invalid_pixels_excluded': True
    }

    # Save detailed metrics to file
    with open(os.path.join(OUTPUT_PATH, f'detailed_masked_metrics_cosine_{method}.json'), 'w') as f:
        json.dump(detailed_metrics, f, indent=4)

    print("Detailed masked metrics saved!")
else:
    print(" Could not calculate detailed metrics - no valid data found!")

# Save model
best_model.save(os.path.join(OUTPUT_PATH, f'flood_resunet_model_masked_cosine_{method}.keras'))

print("Model and metrics saved!")
print(f"All outputs are saved to: {OUTPUT_PATH}")
print(f"Cosine Annealing method used: {method}")

print("\n" + "="*80)
print("MASKED TRAINING COMPLETE!")


## Inference

In [ ]:
# Unified Dataset Inference Script
# Tests trained model on 10 samples each from Otsu and Sen1Floods11 datasets

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import models
import matplotlib.pyplot as plt
import json
import pandas as pd
from PIL import Image
import glob
import random
from sklearn.metrics import confusion_matrix
from tensorflow.keras import backend as K

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

print("TensorFlow version:", tf.__version__)
print("GPU Available:", len(tf.config.list_physical_devices('GPU')) > 0)

# ==================== DEFINE SAME CUSTOM FUNCTIONS AS TRAINING ====================

def masked_dice_coefficient(y_true, y_pred, validity_mask, smooth=1e-6):
    """Calculate Dice coefficient only for valid pixels"""
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)
    
    intersection = K.sum(y_true_f * y_pred_f * validity_f)
    union = K.sum(y_true_f * validity_f) + K.sum(y_pred_f * validity_f)
    
    return (2. * intersection + smooth) / (union + smooth)

def masked_dice_loss(y_true_and_validity, y_pred):
    """Masked Dice loss function that ignores invalid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    return 1 - masked_dice_coefficient(y_true, y_pred, validity_mask)

def masked_focal_tversky_loss(y_true_and_validity, y_pred, alpha=0.7, beta=0.3, gamma=1.5, smooth=1e-6):
    """Masked Focal Tversky Loss that ignores invalid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)
    
    true_pos = K.sum(y_true_f * y_pred_f * validity_f)
    false_neg = K.sum(y_true_f * (1 - y_pred_f) * validity_f)
    false_pos = K.sum((1 - y_true_f) * y_pred_f * validity_f)
    
    tversky = (true_pos + smooth) / (true_pos + alpha * false_neg + beta * false_pos + smooth)
    focal_tversky = K.pow((1 - tversky), gamma)
    
    return focal_tversky

def masked_iou_score(y_true_and_validity, y_pred, smooth=1e-6):
    """Calculate IoU score only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)
    
    intersection = K.sum(y_true_f * y_pred_f * validity_f)
    union = K.sum(y_true_f * validity_f) + K.sum(y_pred_f * validity_f) - intersection
    
    return (intersection + smooth) / (union + smooth)

def masked_binary_accuracy(y_true_and_validity, y_pred):
    """Calculate binary accuracy only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_pred_binary = K.cast(y_pred_masked > 0.5, K.floatx())
    
    correct = K.cast(K.equal(y_true_masked, y_pred_binary), K.floatx()) * validity_mask
    total_valid = K.sum(validity_mask)
    
    return K.sum(correct) / (total_valid + K.epsilon())

def masked_f1_score_metric(y_true_and_validity, y_pred, smooth=1e-6):
    """Calculate F1 score only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)

    true_positives = K.sum(y_true_f * y_pred_f * validity_f)
    predicted_positives = K.sum(y_pred_f * validity_f)
    actual_positives = K.sum(y_true_f * validity_f)

    precision = (true_positives + smooth) / (predicted_positives + smooth)
    recall = (true_positives + smooth) / (actual_positives + smooth)

    f1 = 2 * (precision * recall) / (precision + recall + smooth)
    return f1

def masked_precision_metric(y_true_and_validity, y_pred, smooth=1e-6):
    """Calculate precision only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)

    true_positives = K.sum(y_true_f * y_pred_f * validity_f)
    predicted_positives = K.sum(y_pred_f * validity_f)

    precision = (true_positives + smooth) / (predicted_positives + smooth)
    return precision

def masked_recall_metric(y_true_and_validity, y_pred, smooth=1e-6):
    """Calculate recall only for valid pixels"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    
    y_true_masked = y_true * validity_mask
    y_pred_masked = y_pred * validity_mask
    
    y_true_f = K.flatten(y_true_masked)
    y_pred_f = K.flatten(y_pred_masked)
    validity_f = K.flatten(validity_mask)

    true_positives = K.sum(y_true_f * y_pred_f * validity_f)
    actual_positives = K.sum(y_true_f * validity_f)

    recall = (true_positives + smooth) / (actual_positives + smooth)
    return recall

def masked_dice_coefficient_metric(y_true_and_validity, y_pred):
    """Wrapper for masked dice coefficient"""
    y_true = y_true_and_validity[..., 0:1]
    validity_mask = y_true_and_validity[..., 1:2]
    return masked_dice_coefficient(y_true, y_pred, validity_mask)

# ==================== INFERENCE CLASS ====================

class UnifiedDatasetInference:
    def __init__(self, model_path, data_path, output_path, samples_per_dataset=10):
        """
        Initialize inference for unified dataset
        
        Args:
            model_path: Path to trained model
            data_path: Path to preprocessed unified dataset
            output_path: Path to save inference results
            samples_per_dataset: Number of samples to test per dataset type
        """
        self.model_path = model_path
        self.data_path = data_path
        self.output_path = output_path
        self.samples_per_dataset = samples_per_dataset
        
        os.makedirs(output_path, exist_ok=True)
        
        # Load model
        self.model = self.load_model()
        
        # Load dataset info
        self.dataset_info = self.load_dataset_info()
        
        # Select samples
        self.selected_samples = self.select_samples()
        
    def load_model(self):
        """Load trained model with custom objects"""
        print("Loading trained model...")
        
        custom_objects = {
            'masked_dice_coefficient_metric': masked_dice_coefficient_metric,
            'masked_focal_tversky_loss': masked_focal_tversky_loss,
            'masked_iou_score': masked_iou_score,
            'masked_binary_accuracy': masked_binary_accuracy,
            'masked_f1_score_metric': masked_f1_score_metric,
            'masked_precision_metric': masked_precision_metric,
            'masked_recall_metric': masked_recall_metric
        }
        
        try:
            model = models.load_model(self.model_path, custom_objects=custom_objects)
            print(" Model loaded successfully")
            return model
        except Exception as e:
            print(f" Error loading model: {e}")
            raise

    def load_dataset_info(self):
        """Load unified dataset split information"""
        splits_info_path = os.path.join(self.data_path, 'unified_splits_info.json')
        
        if not os.path.exists(splits_info_path):
            print(f" Dataset info not found: {splits_info_path}")
            raise FileNotFoundError("Unified splits info not found")
        
        with open(splits_info_path, 'r') as f:
            info = json.load(f)
        
        print(" Dataset info loaded successfully")
        print(f"Total Otsu samples: {info['dataset_info']['total_otsu_samples']}")
        print(f"Total Sen1Floods11 samples: {info['dataset_info']['total_sen1floods_samples']}")
        
        return info

    def select_samples(self):
        """Select samples from each dataset type for inference"""
        print(f"\nSelecting {self.samples_per_dataset} samples from each dataset...")
        
        selected = {'otsu': [], 'sen1floods': []}
        
        # Check test split for samples from each dataset
        test_samples = self.dataset_info['splits']['test']
        
        otsu_samples = [s for s in test_samples if s['dataset_type'] == 'otsu']
        sen1floods_samples = [s for s in test_samples if s['dataset_type'] == 'sen1floods']
        
        print(f"Available test samples - Otsu: {len(otsu_samples)}, Sen1Floods11: {len(sen1floods_samples)}")
        
        # Randomly select samples
        if len(otsu_samples) >= self.samples_per_dataset:
            selected['otsu'] = random.sample(otsu_samples, self.samples_per_dataset)
        else:
            selected['otsu'] = otsu_samples
            print(f"  Only {len(otsu_samples)} Otsu samples available")
        
        if len(sen1floods_samples) >= self.samples_per_dataset:
            selected['sen1floods'] = random.sample(sen1floods_samples, self.samples_per_dataset)
        else:
            selected['sen1floods'] = sen1floods_samples
            print(f"  Only {len(sen1floods_samples)} Sen1Floods11 samples available")
        
        print(f"Selected samples - Otsu: {len(selected['otsu'])}, Sen1Floods11: {len(selected['sen1floods'])}")
        
        return selected

    def load_sample_data(self, split='test'):
        """Load actual image and mask data for selected samples"""
        print("Loading sample data...")
        
        sample_data = {'otsu': [], 'sen1floods': []}
        
        # Get all test files
        img_files = sorted(glob.glob(os.path.join(self.data_path, split, 'images', '*.npy')))
        mask_files = sorted(glob.glob(os.path.join(self.data_path, split, 'masks', '*.npy')))
        validity_files = sorted(glob.glob(os.path.join(self.data_path, split, 'validity_masks', '*.npy')))
        
        if len(img_files) == 0:
            raise ValueError(f"No image files found in {self.data_path}/{split}/images/")
        
        # Map sample indices to actual files
        all_test_samples = self.dataset_info['splits']['test']
        
        for dataset_type in ['otsu', 'sen1floods']:
            for sample_info in self.selected_samples[dataset_type]:
                # Find the index of this sample in the test split
                sample_idx = None
                for idx, test_sample in enumerate(all_test_samples):
                    if (test_sample['dataset_type'] == sample_info['dataset_type'] and 
                        test_sample['base_id'] == sample_info['base_id']):
                        sample_idx = idx
                        break
                
                if sample_idx is not None and sample_idx < len(img_files):
                    # Load the actual data
                    img_data = np.load(img_files[sample_idx])
                    mask_data = np.load(mask_files[sample_idx])
                    validity_data = np.load(validity_files[sample_idx])
                    
                    sample_data[dataset_type].append({
                        'image': img_data,
                        'mask': mask_data,
                        'validity': validity_data,
                        'info': sample_info,
                        'file_idx': sample_idx
                    })
        
        print(f"Loaded data - Otsu: {len(sample_data['otsu'])}, Sen1Floods11: {len(sample_data['sen1floods'])}")
        
        return sample_data

    def run_inference(self, sample_data):
        """Run inference on selected samples"""
        print("Running inference...")
        
        results = {'otsu': [], 'sen1floods': []}
        
        for dataset_type in ['otsu', 'sen1floods']:
            print(f"Processing {dataset_type} samples...")
            
            for sample in sample_data[dataset_type]:
                # Prepare input
                img = sample['image']
                img_batch = np.expand_dims(img, axis=0)  # Add batch dimension
                
                # Run prediction
                pred = self.model.predict(img_batch, verbose=0)[0].squeeze()
                
                # Calculate metrics
                gt_mask = sample['mask']
                validity_mask = sample['validity']
                
                metrics = self.calculate_sample_metrics(gt_mask, pred, validity_mask)
                
                results[dataset_type].append({
                    'image': img,
                    'ground_truth': gt_mask,
                    'validity_mask': validity_mask,
                    'prediction': pred,
                    'metrics': metrics,
                    'info': sample['info'],
                    'file_idx': sample['file_idx']
                })
        
        return results

    def calculate_sample_metrics(self, gt_mask, pred, validity_mask, threshold=0.5):
        """Calculate metrics for a single sample"""
        # Apply validity mask
        valid_pixels = validity_mask > 0.5
        invalid_pixels = ~valid_pixels
        
        if np.sum(valid_pixels) == 0:
            return {
                'dice': 0.0, 'iou': 0.0, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0,
                'accuracy': 0.0, 'valid_pixels': 0, 'invalid_pixels': int(np.sum(invalid_pixels)),
                'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0
            }
        
        # Extract valid pixels only
        gt_valid = gt_mask[valid_pixels]
        pred_valid = pred[valid_pixels]
        
        # Apply threshold
        pred_binary = (pred_valid > threshold).astype(np.int32)
        gt_binary = (gt_valid > threshold).astype(np.int32)
        
        # Calculate confusion matrix
        try:
            cm = confusion_matrix(gt_binary, pred_binary, labels=[0, 1])
            if cm.shape == (2, 2):
                tn, fp, fn, tp = cm.ravel()
            else:
                tn = fp = fn = tp = 0
        except:
            tn = fp = fn = tp = 0
        
        # Calculate metrics
        intersection = tp
        union = tp + fp + fn
        
        dice = (2 * intersection + 1e-8) / (2 * intersection + fp + fn + 1e-8)
        iou = (intersection + 1e-8) / (union + 1e-8)
        precision = (tp + 1e-8) / (tp + fp + 1e-8)
        recall = (tp + 1e-8) / (tp + fn + 1e-8)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
        accuracy = (tp + tn + 1e-8) / (tp + tn + fp + fn + 1e-8)
        
        return {
            'dice': float(dice),
            'iou': float(iou),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1),
            'accuracy': float(accuracy),
            'valid_pixels': int(np.sum(valid_pixels)),
            'invalid_pixels': int(np.sum(invalid_pixels)),
            'tp': int(tp), 'fp': int(fp), 'tn': int(tn), 'fn': int(fn)
        }

    def visualize_results(self, results):
        """Create comprehensive visualizations"""
        print("Creating visualizations...")
        
        # Calculate total samples for visualization
        total_otsu = len(results['otsu'])
        total_sen1floods = len(results['sen1floods'])
        total_samples = total_otsu + total_sen1floods
        
        if total_samples == 0:
            print("❌ No samples to visualize")
            return
        
        # Create figure
        fig_height = 6 * total_samples
        fig, axes = plt.subplots(total_samples, 6, figsize=(30, fig_height))
        
        if total_samples == 1:
            axes = axes.reshape(1, -1)
        
        row = 0
        
        # Plot Otsu samples
        for i, sample in enumerate(results['otsu']):
            self.plot_sample_row(axes, row, sample, 'Otsu', i)
            row += 1
        
        # Plot Sen1Floods11 samples
        for i, sample in enumerate(results['sen1floods']):
            self.plot_sample_row(axes, row, sample, 'Sen1Floods11', i)
            row += 1
        
        plt.tight_layout()
        plt.savefig(os.path.join(self.output_path, 'inference_results.png'), dpi=300, bbox_inches='tight')
        plt.show()

    def plot_sample_row(self, axes, row, sample, dataset_name, sample_idx):
        """Plot a single sample across 6 columns"""
        img = sample['image']
        gt = sample['ground_truth']
        validity = sample['validity_mask']
        pred = sample['prediction']
        metrics = sample['metrics']
        
        # Normalize image for visualization
        img_viz = (img - img.min()) / (img.max() - img.min() + 1e-8)
        
        # Create RGB composite
        rgb_viz = np.stack([
            img_viz[:, :, 0],  # VV as R
            img_viz[:, :, 1],  # NewBand1 as G  
            img_viz[:, :, 2]   # NewBand2 as B
        ], axis=-1)
        rgb_viz = np.clip(rgb_viz, 0, 1)
        
        valid_pixels = validity > 0.5
        invalid_pixels = ~valid_pixels
        
        # 1. SAR Composite
        axes[row, 0].imshow(rgb_viz)
        axes[row, 0].set_title(f'{dataset_name} Sample {sample_idx+1}\nSAR Composite\n(VV, NewBand1, NewBand2)')
        axes[row, 0].axis('off')
        
        # 2. Ground Truth with invalid areas
        gt_display = np.ones_like(gt) * 0.5  # Gray for invalid
        gt_display[valid_pixels] = gt[valid_pixels]
        axes[row, 1].imshow(gt_display, cmap='Blues', vmin=0, vmax=1)
        axes[row, 1].set_title(f'Ground Truth\n(Gray = Invalid)\nValid: {metrics["valid_pixels"]:,} px')
        axes[row, 1].axis('off')
        
        # 3. Validity Mask
        axes[row, 2].imshow(validity, cmap='RdYlGn', vmin=0, vmax=1)
        invalid_pct = 100 * metrics['invalid_pixels'] / (metrics['valid_pixels'] + metrics['invalid_pixels'])
        axes[row, 2].set_title(f'Validity Mask\nInvalid: {invalid_pct:.1f}%\nDataset: {dataset_name}')
        axes[row, 2].axis('off')
        
        # 4. Model Prediction
        axes[row, 3].imshow(pred, cmap='Blues', vmin=0, vmax=1)
        axes[row, 3].set_title(f'Model Prediction\n(All pixels)')
        axes[row, 3].axis('off')
        
        # 5. Masked Prediction
        pred_masked = pred.copy()
        pred_masked[invalid_pixels] = 0.5  # Gray out invalid
        axes[row, 4].imshow(pred_masked, cmap='Blues', vmin=0, vmax=1)
        axes[row, 4].set_title(f'Masked Prediction\n(Valid pixels only)')
        axes[row, 4].axis('off')
        
        # 6. Metrics Display
        axes[row, 5].text(0.1, 0.9, f'Dataset: {dataset_name}', fontsize=12, weight='bold', transform=axes[row, 5].transAxes)
        axes[row, 5].text(0.1, 0.8, f'IoU: {metrics["iou"]:.3f}', fontsize=11, transform=axes[row, 5].transAxes)
        axes[row, 5].text(0.1, 0.7, f'Dice: {metrics["dice"]:.3f}', fontsize=11, transform=axes[row, 5].transAxes)
        axes[row, 5].text(0.1, 0.6, f'F1: {metrics["f1"]:.3f}', fontsize=11, transform=axes[row, 5].transAxes)
        axes[row, 5].text(0.1, 0.5, f'Precision: {metrics["precision"]:.3f}', fontsize=11, transform=axes[row, 5].transAxes)
        axes[row, 5].text(0.1, 0.4, f'Recall: {metrics["recall"]:.3f}', fontsize=11, transform=axes[row, 5].transAxes)
        axes[row, 5].text(0.1, 0.3, f'Accuracy: {metrics["accuracy"]:.3f}', fontsize=11, transform=axes[row, 5].transAxes)
        
        # Confusion matrix info
        axes[row, 5].text(0.1, 0.2, f'TP: {metrics["tp"]}, FP: {metrics["fp"]}', fontsize=10, transform=axes[row, 5].transAxes)
        axes[row, 5].text(0.1, 0.1, f'TN: {metrics["tn"]}, FN: {metrics["fn"]}', fontsize=10, transform=axes[row, 5].transAxes)
        
        axes[row, 5].set_xlim(0, 1)
        axes[row, 5].set_ylim(0, 1)
        axes[row, 5].axis('off')

    def calculate_dataset_statistics(self, results):
        """Calculate and display statistics for each dataset"""
        print("\n" + "="*80)
        print("📊 DATASET-SPECIFIC INFERENCE RESULTS")
        print("="*80)
        
        stats = {}
        
        for dataset_type in ['otsu', 'sen1floods']:
            if len(results[dataset_type]) == 0:
                continue
                
            metrics_list = [sample['metrics'] for sample in results[dataset_type]]
            
            # Calculate averages
            avg_metrics = {}
            for metric in ['dice', 'iou', 'f1', 'precision', 'recall', 'accuracy']:
                avg_metrics[metric] = np.mean([m[metric] for m in metrics_list])
            
            # Calculate totals
            total_tp = sum(m['tp'] for m in metrics_list)
            total_fp = sum(m['fp'] for m in metrics_list)
            total_tn = sum(m['tn'] for m in metrics_list)
            total_fn = sum(m['fn'] for m in metrics_list)
            total_valid = sum(m['valid_pixels'] for m in metrics_list)
            total_invalid = sum(m['invalid_pixels'] for m in metrics_list)
            
            # Overall metrics from combined confusion matrix
            overall_precision = (total_tp + 1e-8) / (total_tp + total_fp + 1e-8)
            overall_recall = (total_tp + 1e-8) / (total_tp + total_fn + 1e-8)
            overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall + 1e-8)
            overall_accuracy = (total_tp + total_tn + 1e-8) / (total_tp + total_tn + total_fp + total_fn + 1e-8)
            overall_iou = (total_tp + 1e-8) / (total_tp + total_fp + total_fn + 1e-8)
            
            stats[dataset_type] = {
                'sample_count': len(results[dataset_type]),
                'average_metrics': avg_metrics,
                'overall_metrics': {
                    'precision': float(overall_precision),
                    'recall': float(overall_recall), 
                    'f1': float(overall_f1),
                    'accuracy': float(overall_accuracy),
                    'iou': float(overall_iou)
                },
                'confusion_matrix': {
                    'tp': total_tp, 'fp': total_fp, 'tn': total_tn, 'fn': total_fn
                },
                'pixel_stats': {
                    'total_valid': total_valid,
                    'total_invalid': total_invalid,
                    'invalid_percentage': 100 * total_invalid / (total_valid + total_invalid)
                }
            }
            
            dataset_display = 'S1F11-Otsu' if dataset_type == 'otsu' else 'Sen1Floods11'
            print(f"\n {dataset_display} Dataset Results ({len(results[dataset_type])} samples):")
            print(f"{'─'*50}")
            print(f"Average Per-Sample Metrics:")
            print(f"  IoU: {avg_metrics['iou']:.4f}")
            print(f"  Dice: {avg_metrics['dice']:.4f}")
            print(f"  F1: {avg_metrics['f1']:.4f}")
            print(f"  Precision: {avg_metrics['precision']:.4f}")
            print(f"  Recall: {avg_metrics['recall']:.4f}")
            print(f"  Accuracy: {avg_metrics['accuracy']:.4f}")
            
            print(f"\nOverall Metrics (Combined Confusion Matrix):")
            print(f"  IoU: {overall_iou:.4f}")
            print(f"  F1: {overall_f1:.4f}")
            print(f"  Precision: {overall_precision:.4f}")
            print(f"  Recall: {overall_recall:.4f}")
            print(f"  Accuracy: {overall_accuracy:.4f}")
            
            print(f"\nPixel Statistics:")
            print(f"  Valid pixels: {total_valid:,}")
            print(f"  Invalid pixels: {total_invalid:,} ({stats[dataset_type]['pixel_stats']['invalid_percentage']:.1f}%)")
            
            print(f"\nConfusion Matrix:")
            print(f"  TP: {total_tp:,}, FP: {total_fp:,}")
            print(f"  TN: {total_tn:,}, FN: {total_fn:,}")
        
        # Compare datasets
        if 'otsu' in stats and 'sen1floods' in stats:
            print(f"\n DATASET COMPARISON:")
            print(f"{'─'*50}")
            
            otsu_iou = stats['otsu']['overall_metrics']['iou']
            sen1floods_iou = stats['sen1floods']['overall_metrics']['iou']
            iou_diff = otsu_iou - sen1floods_iou
            
            print(f"IoU Difference (Otsu - Sen1Floods11): {iou_diff:+.4f}")
            if abs(iou_diff) < 0.01:
                print("→ Similar performance on both datasets")
            elif iou_diff > 0:
                print("→ Better performance on Otsu dataset")
            else:
                print("→ Better performance on Sen1Floods11 dataset")
            
            otsu_invalid = stats['otsu']['pixel_stats']['invalid_percentage']
            sen1floods_invalid = stats['sen1floods']['pixel_stats']['invalid_percentage']
            print(f"Invalid pixel % - Otsu: {otsu_invalid:.1f}%, Sen1Floods11: {sen1floods_invalid:.1f}%")
        
        print("="*80)
        
        return stats

    def save_results(self, results, stats):
        """Save detailed results to files"""
        print("Saving results...")
        
        # Save individual sample results
        all_results = []
        for dataset_type in ['otsu', 'sen1floods']:
            for i, sample in enumerate(results[dataset_type]):
                result_row = {
                    'dataset': dataset_type,
                    'sample_idx': i,
                    'file_idx': sample['file_idx'],
                    **sample['metrics']
                }
                all_results.append(result_row)
        
        # Save to CSV
        df = pd.DataFrame(all_results)
        df.to_csv(os.path.join(self.output_path, 'inference_results.csv'), index=False)
        
        # Save summary statistics
        with open(os.path.join(self.output_path, 'inference_statistics.json'), 'w') as f:
            json.dump(stats, f, indent=4)
        
        print(f"✅ Results saved to {self.output_path}")

    def run_complete_inference(self):
        """Run the complete inference pipeline"""
        print("\n STARTING UNIFIED DATASET INFERENCE")
        
        # Load sample data
        sample_data = self.load_sample_data()
        
        # Run inference
        results = self.run_inference(sample_data)
        
        # Calculate statistics
        stats = self.calculate_dataset_statistics(results)
        
        # Create visualizations
        self.visualize_results(results)
        
        # Save results
        self.save_results(results, stats)
        
        print("\n✅ INFERENCE COMPLETE!")
        return results, stats

# ==================== MAIN EXECUTION ====================

if __name__ == "__main__":
    # Configuration - UPDATE THESE PATHS
    MODEL_PATH = "/kaggle/working/united_SEN1FLOODS11/best_model_masked_cosine_annealing.keras"  # Update this
    DATA_PATH = "/kaggle/working/unified_preprocessed"  # Update this
    OUTPUT_PATH = "/kaggle/working/inference_results"
    SAMPLES_PER_DATASET = 10
    
    print(" UNIFIED DATASET INFERENCE SETUP")
    print("="*50)
    print(f"Model path: {MODEL_PATH}")
    print(f"Data path: {DATA_PATH}")
    print(f"Output path: {OUTPUT_PATH}")
    print(f"Samples per dataset: {SAMPLES_PER_DATASET}")
    
    # Check if paths exist
    if not os.path.exists(MODEL_PATH):
        print(f" Model not found: {MODEL_PATH}")
        print("Please update MODEL_PATH to your trained model")
        exit(1)
    
    if not os.path.exists(DATA_PATH):
        print(f" Data not found: {DATA_PATH}")
        print("Please update DATA_PATH to your preprocessed data")
        exit(1)
    
    # Initialize and run inference
    inference = UnifiedDatasetInference(
        model_path=MODEL_PATH,
        data_path=DATA_PATH,
        output_path=OUTPUT_PATH,
        samples_per_dataset=SAMPLES_PER_DATASET
    )
    
    # Run complete inference
    results, stats = inference.run_complete_inference()
    
    print(f"\n All outputs saved to: {OUTPUT_PATH}")
    print(" Files created:")
    print("  - inference_results.png (visualizations)")
    print("  - inference_results.csv (detailed metrics)")
    print("  - inference_statistics.json (summary stats)")

    # fixed msg